In [31]:
import pandas as pd
import importlib   # Import the importlib module
import Create_Zoo
import random
import plotly.express as px
import datetime
import string
from datetime import timedelta


In [48]:

# Reload the module after making changes to the .py file
importlib.reload(Create_Zoo)
# 'Enclosures', 'Staff', 'Animals','Invoices','Budgets','Calendar','Visits'

df_Enclosures = Create_Zoo.createEntity('Enclosures')
df_Staff = Create_Zoo.createEntity('Staff')
df_Animals = Create_Zoo.createEntity("Animals")
df_Invoices = Create_Zoo.createEntity('Invoices')
df_Budgets = Create_Zoo.createEntity('Budgets')
df_Calendar = Create_Zoo.createEntity('Calendar', verbose = True)
df_Visits = Create_Zoo.createEntity("Visits", verbose = True)

 ... Creating df for Enclosures
 ... Creating df for Staff
 ... Creating df for Animals
 ... Creating df for Invoices
 ... Creating df for Budgets
 ... Creating df for Calendar
 ... Created df for Calendar with 366 rows and 8 cols
 ... columns are Index(['day_of_year', 'date', 'day_of_week', 'type', 'weekend', 'bank_holiday',
       'school_holiday', 'visits'],
      dtype='object')
 ... Creating df for Visits
 ... Created df for Visits with 275062 rows and 3 cols
 ... columns are Index(['Date', 'Enclosure_ID', 'Member_ID'], dtype='object')


# chart

In [49]:
print(df_Calendar['visits'].sum())
print(df_Visits.shape[0])

275095
275062


In [50]:
# Create an interactive line chart
fig = px.line(df_Calendar, x="date", y="visits", title="Visits Over the Year 2024",
              labels={"date": "Date", "visits": "Number Of Visits", "weekend": "Weekend"},
              hover_data={"weekend": True} )
    
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Visits",
    template="plotly_white",  # Optional: cleaner template
    xaxis=dict(rangeslider=dict(visible=True))  # Enable range slider
)

# Mark Bank Holidays
for date in df_Calendar[df_Calendar["type"] != 'Normal Day']["date"]:

    date1 = datetime.datetime.strptime(date, '%Y-%m-%d')
    date2 = date1 + timedelta(days=1)
    date2 = date2.strftime("%Y-%m-%d")
    
    fig.add_vrect(
        x0=date,
        x1=date2,
        label=dict(
            text="Bank Holiday",
            textposition="top center",
        ),
        fillcolor="green",
        opacity=0.25,
        line_width=0,
    )

# Customize the range slider with a light grey background
fig.update_layout(
    template="plotly_white",
    xaxis=dict(
        title="Date Range Slider",
        rangeslider=dict(
            visible=True,         # Enable the range slider
            bgcolor="lightgrey",  # Set the background color of the slider
        ),
    ),
    yaxis=dict(title="Number of Visits"),
)

fig.show()

In [53]:
# Example DataFrame
# Ensure the 'date' column is of datetime type
df_Visits['Date'] = pd.to_datetime(df_Visits['Date'])

# Group by date and count records
df_summary = df_Visits.groupby('Date').size().reset_index(name='total_records')

# Create the Plotly chart
fig = px.line(df_summary, x='Date', y='total_records', title='Total Records per Date',
              labels={'date': 'Date', 'total_records': 'Total Records'})

# Show the chart
fig.show()

In [ ]:
df_Calendar.head()

In [ ]:
total_visits = df_Calendar['visits'].sum()


print(df_Targets[0:3])

In [20]:
# generate the targets and calculate the proportion for each
# enclosure, and the split between ad-hoc visits and member visits
dict_Targets = Create_Zoo.generate_visit_targets()
df_Targets = pd.DataFrame(dict_Targets)
df_Targets['Total'] = df_Targets['Visits'] + df_Targets['MemberVisits']

df_Targets["prop"] = df_Targets['Total'] / df_Targets['Total'].sum()
df_Targets['r1'] = df_Targets["Visits"] / df_Targets['Total']
df_Targets['r2'] = df_Targets["MemberVisits"] / df_Targets['Total']

# get the member_ids
dict_Members = Create_Zoo.generate_members()

# List to return
visits =[]

# for each day in the calender
for i, row in df_Calendar.iterrows():
    date = row['date']
    total_today = row['visits']
    total_visits_created = 0

    # for each enclosure target
    for j, row_targ in df_Targets.iterrows():
        enc = row_targ['Enclosure_ID']
        prop = row_targ['prop']
        if prop == 0: continue
        t1 = row_targ["r1"]
        t2 = row_targ["r2"]

        # visits for the enclosure
        total_enclosure = round(total_today * prop, 0)
        v1 = round(total_enclosure * t1) # non-members
        v2 = round(total_enclosure * t2) # members

        # ad-hoc visits
        for visit in range(0,v1):
            visit_nm = {"Date": date, 
                        "Enclosure_ID": enc,
                        "Member_ID":'None'}
            visits.append(visit_nm)

        # member visits
        for visit in range(0,v2):
            mem_id = random.choice(dict_Members)
            mem_id = mem_id['Member_ID']
            visit_mem = {"Date": date, 
                        "Enclosure_ID": enc,
                        "Member_ID":mem_id}
            visits.append(visit_mem)
            
    return(visits)
